<a href="https://colab.research.google.com/github/Tejaswini1309/Satellite-Imagery-Based-Property-Valuation/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
torch.cuda.is_available()


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/drive/MyDrive/project_cdc/train_images.zip /content/train.zip
!unzip -q /content/train.zip -d /content/train_data
!ls /content/train_data | head -n 5
!ls /content/train_data | wc -l

Mounted at /content/drive
1000102.png
1001200050.png
1003000175.png
100300280.png
100300530.png
16110


In [3]:
!cp "/content/drive/MyDrive/project_cdc/train1.csv" /content/train1.csv
!cp "/content/drive/MyDrive/project_cdc/test2.csv" /content/test2.csv

In [4]:
#to check if all the folders were pushed or not
!ls /content


drive  sample_data  test2.csv  train1.csv  train_data  train.zip


In [5]:
# loading the data into df using pandas and df.shape[0] gives the number of training examples we have
import pandas as pd
df = pd.read_csv("/content/train1.csv")
df.shape[0]

16209

In [6]:
df.shape[0]


16209

In [7]:
# gives a breif statistycal analysis of all the images we have
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,1.620900e+04,1.620900e+04,16209.00000,16209.000000,16209.000000,1.620900e+04,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000,16209.000000
mean,4.575771e+09,5.374703e+05,3.36782,2.113054,2073.274601,1.486767e+04,1.498828,0.006971,0.234253,3.407860,7.652971,1784.754396,288.520205,1971.152755,82.738108,98077.974767,47.560707,-122.214003,1983.152261,12735.572707
std,2.874661e+09,3.603036e+05,0.93327,0.765242,907.009491,3.882570e+04,0.543032,0.083206,0.763152,0.651553,1.171050,821.820844,438.598910,29.372698,397.861148,53.355282,0.138340,0.140093,681.905161,26933.162012
min,1.000102e+06,7.500000e+04,0.00000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.159300,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.200000e+05,3.00000,1.500000,1430.000000,5.004000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1200.000000,0.000000,1952.000000,0.000000,98033.000000,47.472500,-122.328000,1480.000000,5098.000000
50%,3.904950e+09,4.500000e+05,3.00000,2.250000,1910.000000,7.599000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.572400,-122.230000,1840.000000,7620.000000
75%,7.304301e+09,6.400000e+05,4.00000,2.500000,2550.000000,1.063100e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2200.000000,560.000000,1997.000000,0.000000,98117.000000,47.678200,-122.125000,2360.000000,10053.000000
max,9.900000e+09,7.700000e+06,33.00000,8.000000,12050.000000,1.164794e+06,3.500000,1.000000,4.000000,5.000000,13.000000,8860.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [8]:
# gives list of columns ( the feature names )
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [9]:
# while all other features are either float or int type date is a object type of this format ___ which shows the date , time
# of when the house was sold , so we extract the month and year from it and ommit the other parts as they arent that significant
# in determining price ( like the time when it was sold )

df['date']=pd.to_datetime(df['date'],format = '%Y%m%dT%H%M%S')
df['year_sold'] = df['date'].dt.year
df['month_sold'] = df['date'].dt.month

In [10]:
# this is to make a new feature called house_age which shows us how old the house was when it was sold
df['house_age'] = df['year_sold']-df['yr_built']
df.drop(columns=['date'],inplace=True)

In [11]:
df.columns

Index(['id', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year_sold', 'month_sold', 'house_age'],
      dtype='object')

In [12]:
# these are features of our tabular info which have to be scaled ( id is omitted as it doseny say us much about the price of the houses and
# prices too as they are our targer , waterfront is already scaled , zipcode will be embedded as its not a feature that we can define with
# its magnitude zipcode 96023 > 96543 we cannot define it this way hence later we will label the zipcodes and use embedding )
num_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated','lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year_sold', 'month_sold', 'house_age']

In [13]:
# labelling the zipcodes , we have 71 zipcodes so labeling them 0 to 70
# also we have diff zipcodes in our test data so adding unk so that it treats new variables as unknown but dosent lead to an error
from sklearn.preprocessing import LabelEncoder
le_zip = LabelEncoder()
zips = df['zipcode'].astype(str)
zips = pd.concat([zips,pd.Series(['UNK'])])
le_zip.fit(zips)
df['zipcode'] = le_zip.fit_transform(df['zipcode'].astype(str))

In [14]:
import joblib
joblib.dump(le_zip,"/content/drive/MyDrive/project_cdc/zipcode_encoder1.pkl")

['/content/drive/MyDrive/project_cdc/zipcode_encoder1.pkl']

In [15]:
# making val and train datasets by splitting df ( the whole train1.csv ) , we are giving val 20% of the whole trainging dataset
from sklearn.model_selection import train_test_split
df_train , df_val = train_test_split(df,test_size=0.2,random_state=42)

In [16]:
# scaling the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[num_features] = scaler.fit_transform(df_train[num_features])
df_val[num_features] = scaler.transform(df_val[num_features])

In [17]:
price_scaler = StandardScaler()
df_train["price"] = price_scaler.fit_transform(
    df_train[["price"]]
)
df_val["price"] = price_scaler.transform(
    df_val[["price"]]
)


In [18]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406] ,std = [0.229, 0.224, 0.225])
])

In [19]:
# in this dataset class we get our data ready for the dataloader to split them into batches
# images are converted from png to pil , we use 224x224 images itself so no need to resize them
# tabular features are converted into float32 dtype along with the price
# zipcode is converted to long type to pass it through embedding
# this data set class on calling __getitem__ returns us the dtype changed price and x_tab , this seperated them from df
# and gives us png images turned into tensors which are scaled and normalised and ready to be passed into resnet along with
# zipcode which can be passed into embedding directly
import os
import torch
from torch.utils.data import Dataset
from PIL import Image

class HouseDataset(Dataset):
    def __init__(self, df, image_dir, features, transform):
        self.df = df
        self.image_dir = image_dir
        self.features = features
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]


        x_tab = torch.tensor(
            row[self.features].values,
            dtype=torch.float32
        )

        zipcode = torch.tensor(
            int(row["zipcode"]),
            dtype=torch.long
        )

        y = torch.tensor(row['price'], dtype=torch.float32)


        house_id = str(int(row["id"]))
        img_path = os.path.join(self.image_dir, f"{house_id}.png")

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return {
            "tabular": x_tab,
            "zipcode": zipcode,
            "image": image,
            "target": y
        }

In [20]:
# these are the list of features in our x_tab ( tabular set of features)
num_features=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront','view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated','lat', 'long',
       'sqft_living15', 'sqft_lot15', 'year_sold', 'month_sold', 'house_age']

In [21]:
# passing train and val data sets through dataset class
train_dataset = HouseDataset(df_train,"/content/train_data",num_features,transform)
val_dataset = HouseDataset(df_val,"/content/train_data",num_features,transform)

In [22]:
# an example of how the data looks after passing through the dataset class
train_dataset[10]

{'tabular': tensor([-0.4051, -0.4713, -0.3565,  0.0435, -0.9159,  0.0000, -0.3048, -0.6257,
         -0.5582, -0.5540,  0.3084,  0.2320, -0.2111,  1.3047,  0.8949, -0.3460,
          0.1149, -0.6895,  1.4212, -0.2430]),
 'zipcode': tensor(36),
 'image': tensor([[[-1.6555, -1.3815, -1.5185,  ..., -1.1075, -0.6965, -0.4054],
          [-1.3815, -1.6555, -1.7925,  ..., -0.6965, -0.4226, -0.1314],
          [-1.2103, -1.6727, -1.7925,  ..., -0.4054, -0.0287, -0.0116],
          ...,
          [-1.9295, -1.7925, -1.3987,  ..., -1.5014, -1.5014, -1.3815],
          [-1.2445, -1.1075, -0.8335,  ..., -0.6965, -0.9705, -0.5424],
          [-1.1075, -1.1075, -1.2445,  ..., -0.1486, -0.2856, -0.2684]],
 
         [[-1.4055, -1.2479, -1.2829,  ..., -0.5826, -0.1625, -0.0224],
          [-1.2654, -1.4055, -1.5630,  ..., -0.2850,  0.1176,  0.2752],
          [-0.9853, -1.5455, -1.5455,  ..., -0.0224,  0.2577,  0.3978],
          ...,
          [-1.4055, -1.4230, -0.8627,  ..., -1.4055, -1.4055, -1.2

In [23]:
df_train.shape[0]

12967

In [24]:
# passing the transformed data through dataloader class to convert into batches and shuffle is true so that each batch can be
# representative of the whole data
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset,batch_size=16,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=16,shuffle=False)

In [25]:
# importing resnet18 for images
import torchvision.models as models
resnet = models.resnet18(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 190MB/s]


In [26]:
# checking if gpu is available
torch.cuda.is_available()
torch.cuda.get_device_name(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
# freezing the feature embeddings part of resnet
for param in resnet.parameters():
    param.requires_grad = False
resnet = resnet.to(device)


In [28]:
# changing the fc layer of resnet to a ann as defined below with two layers input being of size 512 , that is how many features you get
# from the feature embeddings part of resnet  , then passing it through layer1 ( Linear ) with activation function of rely , using dropout
# to avoid overfitting of the model , linear layer again which outputs 128 features
import torch.nn as nn
num_f=resnet.fc.in_features
resnet.fc = nn.Sequential(
    nn.Linear(num_f,256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256,128),
    nn.ReLU(),
    nn.Linear(128,64)
)

In [29]:
# using this class we pass the images tensor through resnet , get zipcode embeddings and pass x_tab through a nn , after getting the output
# features of all these we combine them ( 128+10+32) and pass them through a final nn called final_mlp
class Model(nn.Module):
  def __init__(self,resnet,num_tab_features,num_zipcodes):
    super().__init__()
    self.resnet = resnet
    self.zipcode_embedding = nn.Embedding(num_zipcodes,10)
    self.nn_tab = nn.Sequential(
        nn.Linear(num_tab_features,64),
        nn.ReLU(),
        nn.BatchNorm1d(64),
        nn.Dropout(0.2),
        nn.Linear(64,32),
        nn.ReLU()
    )
    self.final_mlp = nn.Sequential(
        nn.Linear(64+10+32,64),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(64,1)
    )
  def forward(self,x_tab,zipcode,image):
    img_feat = self.resnet(image)
    zip_feat = self.zipcode_embedding(zipcode)
    tab_feat = self.nn_tab(x_tab)

    x = torch.cat([img_feat, zip_feat, tab_feat], dim=1)
    return self.final_mlp(x).squeeze(1)




In [30]:
# to get the number of unique zipcodes
num_zipcodes = df["zipcode"].nunique()

In [31]:
# an instance of model class , model object
model = Model(
    resnet=resnet,
    num_tab_features=len(num_features),
    num_zipcodes=num_zipcodes
).to(device)

In [32]:
# we use MSE loss , adam optimiser with alpha ( learning rate as 0.0001)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-5
)
epochs = 15

In [33]:
# to check the size of our outputs from dataloader
batch = next(iter(train_loader))
print(batch["tabular"].shape)
print(batch["image"].shape)
print(batch["zipcode"].shape)
print(batch["target"].shape)

torch.Size([16, 20])
torch.Size([16, 3, 224, 224])
torch.Size([16])
torch.Size([16])


In [55]:
epochs = 2

In [56]:
# training loop , we output the avg , rmse loss of each epoch
for epoch in range(epochs):
  total_loss=0.0
  for batch in train_loader:
    x_tab = batch["tabular"].to(device)
    zipcode = batch["zipcode"].to(device)
    image = batch["image"].to(device)
    y = batch["target"].to(device)
    optimizer.zero_grad()
    y_pred = model(x_tab,zipcode,image)
    loss = loss_function(y_pred , y)
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()
  avg_loss = total_loss/len(train_loader)
  rmse = avg_loss**0.5
  print(f"Epoch {epoch+1} | Train avg: {avg_loss:.2f} | Train RMSE: {rmse*price_scaler.scale_[0]:.2f}")


Epoch 1 | Train avg: 0.08 | Train RMSE: 101498.78
Epoch 2 | Train avg: 0.07 | Train RMSE: 98481.30


In [57]:
# validation error - the error when the prices of the validation set were predicted using the trained model
model.eval()
val_loss=0.0

y_true = []
preds = []
with torch.no_grad():
   for batch in val_loader:
    x_tab = batch["tabular"].to(device)
    zipcode = batch["zipcode"].to(device)
    image = batch["image"].to(device)
    y = batch["target"].to(device)
    y_pred = model(x_tab,zipcode,image)
    loss = loss_function(y_pred , y)
    val_loss+=loss.item()
    y_true.append(y.cpu().numpy())
    preds.append(y_pred.cpu().numpy())
print("Val_loss:", val_loss)
val_rmse = (val_loss / len(val_loader)) ** 0.5
print("Val RMSE:", val_rmse)



Val_loss: 23.48579000029713
Val RMSE: 0.34013754100289084


In [58]:
val_rmse*price_scaler.scale_[0]

np.float64(123056.9650943197)

In [59]:
val_loss/len(val_loader)

0.11569354679949324

In [60]:
import numpy as np

y_true = np.concatenate(y_true)
preds = np.concatenate(preds)

# inverse scale if needed
y_true_real = price_scaler.inverse_transform(y_true.reshape(-1,1)).ravel()
y_pred_real = price_scaler.inverse_transform(preds.reshape(-1,1)).ravel()

r2 = r2_score(y_true_real, y_pred_real)
print("R2:", r2)


R2: 0.879218578338623


In [61]:
# saving the models state so that we can acces it during price prediction
torch.save(
    model.state_dict(),
    "/content/drive/MyDrive/project_cdc/last_model1.pt"
)
